In [1]:
import pandas as pd
import numpy as np

from sklearn import metrics
from tangle.mbspbs10pc import utils

Using TensorFlow backend.


# Load raw data

In [2]:
def data():
    """Test set not returned on purpose."""
    
    labelsfile = '../../tmp/1_METONLY_vs_METX/matched_CEM_table.csv'
    datafile = '../../tmp/item_days_raw_data_.pkl'
    dataset = utils.load_data_labels(datafile, labelsfile)
    padded_mbs_seq, padded_timespan_seq, _ = utils.tokenize(dataset)
    maxlen = padded_mbs_seq.shape[1]

    # Split in training, validation, test sets
    tr_set, v_set, ts_set = utils.train_validation_test_split(
        [padded_mbs_seq, padded_timespan_seq], dataset['Class'],
        test_size=0.4, validation_size=0.1,
        verbose=False, random_state0=1, random_state1=0)

    x_train, y_train = tr_set[0], tr_set[1]
    x_valid, y_valid = v_set[0], v_set[1]
    
    return x_train, y_train, x_valid, y_valid

# Model

In [3]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas.distributions import choice, uniform

def create_model(x_train, y_train, x_valid, y_valid):
    from keras import optimizers as opt
    from keras.preprocessing.sequence import pad_sequences
    from keras.callbacks import EarlyStopping, ReduceLROnPlateau
    from keras.models import Model
    from keras.regularizers import l2
    from keras.layers import LSTM, Bidirectional, Dense, Dot, Dropout, LSTM
    from keras.layers import Embedding, GlobalAveragePooling1D, Input, Multiply
    from tangle.layers import TimespanGuidedNeuralAttention

    # Hyperparameters
    n_recurrent = {{choice([16, 32, 64, 128])}}
    n_attention = {{choice([16, 32, 64, 128])}}
    dr_rate_0 = {{uniform(0, 1)}}
    n_dense = {{choice([16, 32, 64, 128])}}
    dr_rate_1 = {{uniform(0, 1)}}
    lr = {{uniform(1e-5, 1e-1)}}
    embedding_trainable = {{choice([True, False])}}
    batch_size = {{choice([64, 128, 256])}}
    
    # Channel 1: MBS
    mbs_input = Input(shape=(250,), name='mbs_input')
    e = Embedding(2774, 50, name='mbs_embedding')(mbs_input)
    x1 = Bidirectional(LSTM(n_recurrent, return_sequences=True), name='mbs_lstm')(e)

    # Channel 2: Timespans
    timespan_input = Input(shape=(250, 1), name='timespan_input')
    x2 = Bidirectional(LSTM(n_recurrent, return_sequences=True), name='timespan_lstm')(timespan_input)
    
    # Timespan-guided neural attention weights
    alpha = TimespanGuidedNeuralAttention(n_attention, name='tangle_attention')([x1, x2])

    # Combine channels to get contribution and context
    c = Multiply(name='contribution')([alpha, x1])
    x = Dot(axes=1, name='context')([c, e])

    # Output
    x = GlobalAveragePooling1D(name='pooling')(x)
    x = Dropout(dr_rate_0, name='dropout_0')(x) # tune dropout rate
    x = Dense(n_dense, activation='linear', name='fc')(x)
    x = Dropout(dr_rate_1, name='dropout_1')(x)  # tune dropout rate
    output = Dense(1, activation='sigmoid', name='fc_output', activity_regularizer=l2({{uniform(1e-5, 1e-3)}}))(x)

    # Define the model
    model = Model(inputs=[mbs_input, timespan_input],
                  outputs=[output])

    # Compile the model
    model.compile(optimizer=opt.RMSprop(lr=lr),
                  loss='binary_crossentropy',
                  metrics=['acc'])
    
    # Init the embedding matrix
    embedding = pd.read_csv('../../tmp/embedding.50d.csv', header=0, index_col=0)    
    model.get_layer('mbs_embedding').set_weights([embedding.values])
    model.get_layer('mbs_embedding').trainable = embedding_trainable
    
    # Fit model
    model.fit(x_train, y_train, epochs=100,
              callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-6, verbose=1),
                         EarlyStopping(monitor='val_loss', patience=15)] ,
              batch_size=batch_size,
              validation_data=(x_valid, y_valid))
    
    # Evaluate model
    score, acc = model.evaluate(x_valid, y_valid, verbose=0)
    print('score: {}'.format(score))
    print('acc: {}'.format(acc))
    
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [4]:
from hyperas import optim

from keras import optimizers as opt
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.models import Model
from keras.regularizers import l2
from keras.layers import LSTM, Bidirectional, Dense, Dot, Dropout, LSTM
from keras.layers import Embedding, GlobalAveragePooling1D, Input, Multiply
from tangle.layers import TimespanGuidedNeuralAttention


best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=50,
                                      trials=Trials(),
                                      notebook_name='hyperparam_tuning')

>>> Imports:
#coding=utf-8

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from sklearn import metrics
except:
    pass

try:
    from tangle.mbspbs10pc import utils
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from keras import optimizers as opt
except:
    pass

try:
    from keras.preprocessing.sequence import pad_sequences
except:
    pass

try:
    from keras.callbacks import EarlyStopping, ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.regularizers import l2
except:
    pass

try:
    from keras.layers import LSTM, Bidirectional, Dense, Dot, Dropout, LSTM
except:
    pass

try:
    from keras.layers import Embedding, GlobalAveragePooling1D, Input, Multiply
except:
    pass

try:
    from tangle.layers import TimespanGuidedNeuralAtte

/home/samu/miniconda2/lib/python2.7/site-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Train on 6341 samples, validate on 705 samples
Epoch 1/100
6341/6341 [==============================] - 85s 13ms/step - loss: 0.8963 - acc: 0.5567 - val_loss: 0.6921 - val_acc: 0.6184
Epoch 2/100
6341/6341 [==============================] - 82s 13ms/step - loss: 0.6593 - acc: 0.6529 - val_loss: 0.6718 - val_acc: 0.6298
Epoch 3/100
6341/6341 [==============================] - 81s 13ms/step - loss: 0.6255 - acc: 0.6930 - val_loss: 0.6440 - val_acc: 0.6454
Epoch 4/100
6341/6341 [==============================] - 84s 13ms/step - loss: 0.6004 - acc: 0.7035 - val_loss: 0.8186 - val_acc: 0.5745
Epoch 5/100
6341/6341 [==============================] - 75s 12ms/step - loss: 0.5947 - acc: 0.7174 - val_loss: 0.6424 - val_acc: 0.6738
Epoch 6/100
6341/6341 [==============================] - 83s 13ms/step - loss: 0.5725 - acc: 0.7270 - val_loss: 0.7016 - val_acc: 0.5773
Epoch 7/100
6341/6341 [==============================] - 83s 13ms/step - loss: 0.5690 - acc: 0.7305 - val_loss: 0.6358 - val_acc: 0

Epoch 26/100
6341/6341 [==============================] - 159s 25ms/step - loss: 0.6677 - acc: 0.6035 - val_loss: 0.6347 - val_acc: 0.6582

Epoch 00026: ReduceLROnPlateau reducing learning rate to 0.0106321526691.
score: 0.630621042826
acc: 0.658156028369
Train on 6341 samples, validate on 705 samples
Epoch 1/100
6341/6341 [==============================] - 75s 12ms/step - loss: 0.7484 - acc: 0.4931 - val_loss: 0.7056 - val_acc: 0.5418
Epoch 2/100
6341/6341 [==============================] - 75s 12ms/step - loss: 0.7056 - acc: 0.5356 - val_loss: 0.6749 - val_acc: 0.5801
Epoch 3/100
6341/6341 [==============================] - 79s 12ms/step - loss: 0.6871 - acc: 0.5901 - val_loss: 0.6487 - val_acc: 0.6837
Epoch 4/100
6341/6341 [==============================] - 72s 11ms/step - loss: 0.6647 - acc: 0.6319 - val_loss: 0.6767 - val_acc: 0.6071
Epoch 5/100
6341/6341 [==============================] - 72s 11ms/step - loss: 0.6637 - acc: 0.6458 - val_loss: 0.6231 - val_acc: 0.6851
Epoch 6/100


6341/6341 [==============================] - 41s 7ms/step - loss: 0.7997 - acc: 0.5021 - val_loss: 0.7447 - val_acc: 0.4993
Epoch 5/100
6341/6341 [==============================] - 40s 6ms/step - loss: 0.8028 - acc: 0.4976 - val_loss: 0.7390 - val_acc: 0.5007
Epoch 6/100
6341/6341 [==============================] - 39s 6ms/step - loss: 0.8178 - acc: 0.4971 - val_loss: 0.7393 - val_acc: 0.5007
Epoch 7/100
6341/6341 [==============================] - 40s 6ms/step - loss: 0.8047 - acc: 0.4924 - val_loss: 0.7399 - val_acc: 0.5007
Epoch 8/100
6341/6341 [==============================] - 42s 7ms/step - loss: 0.8111 - acc: 0.4912 - val_loss: 0.7402 - val_acc: 0.5007
Epoch 9/100
6341/6341 [==============================] - 41s 6ms/step - loss: 0.8026 - acc: 0.5010 - val_loss: 0.7391 - val_acc: 0.5007
Epoch 10/100
6341/6341 [==============================] - 38s 6ms/step - loss: 0.8230 - acc: 0.4974 - val_loss: 0.7394 - val_acc: 0.5007
Epoch 11/100
6341/6341 [==============================] - 4

...

Epoch 8/100
6341/6341 [==============================] - 151s 24ms/step - loss: 0.4281 - acc: 0.8155 - val_loss: 0.4824 - val_acc: 0.7674
Epoch 9/100
6341/6341 [==============================] - 151s 24ms/step - loss: 0.3979 - acc: 0.8390 - val_loss: 0.6878 - val_acc: 0.6709
Epoch 10/100
6341/6341 [==============================] - 151s 24ms/step - loss: 0.3935 - acc: 0.8343 - val_loss: 0.4910 - val_acc: 0.7688
Epoch 11/100
6341/6341 [==============================] - 150s 24ms/step - loss: 0.3834 - acc: 0.8432 - val_loss: 0.5256 - val_acc: 0.7574
Epoch 12/100
6341/6341 [==============================] - 151s 24ms/step - loss: 0.3596 - acc: 0.8537 - val_loss: 0.5800 - val_acc: 0.7674
Epoch 13/100
6341/6341 [==============================] - 152s 24ms/step - loss: 0.3461 - acc: 0.8601 - val_loss: 0.6598 - val_acc: 0.7220
Epoch 14/100
6341/6341 [==============================] - 151s 24ms/step - loss: 0.3462 - acc: 0.8656 - val_loss: 0.5783 - val_acc: 0.7560
Epoch 15/100
6341/6341 [=====

6341/6341 [==============================] - 514s 81ms/step - loss: 0.3961 - acc: 0.8341 - val_loss: 0.5763 - val_acc: 0.7589
Epoch 14/100
6341/6341 [==============================] - 523s 82ms/step - loss: 0.3756 - acc: 0.8462 - val_loss: 0.5389 - val_acc: 0.7603

Epoch 00014: ReduceLROnPlateau reducing learning rate to 0.0130491117015.
Epoch 15/100
6341/6341 [==============================] - 521s 82ms/step - loss: 0.3157 - acc: 0.8716 - val_loss: 0.5995 - val_acc: 0.7645
Epoch 16/100
6341/6341 [==============================] - 532s 84ms/step - loss: 0.2654 - acc: 0.9011 - val_loss: 0.6673 - val_acc: 0.7560
Epoch 17/100
6341/6341 [==============================] - 516s 81ms/step - loss: 0.2295 - acc: 0.9155 - val_loss: 0.7382 - val_acc: 0.7362
Epoch 18/100
6341/6341 [==============================] - 518s 82ms/step - loss: 0.2203 - acc: 0.9234 - val_loss: 0.7597 - val_acc: 0.7475
Epoch 19/100
6341/6341 [==============================] - 522s 82ms/step - loss: 0.1937 - acc: 0.9333 - 

6341/6341 [==============================] - 534s 84ms/step - loss: 0.0440 - acc: 0.9943 - val_loss: 1.7918 - val_acc: 0.7546
Epoch 23/100
6341/6341 [==============================] - 527s 83ms/step - loss: 0.0421 - acc: 0.9932 - val_loss: 1.8068 - val_acc: 0.7589

Epoch 00023: ReduceLROnPlateau reducing learning rate to 0.00160024105571.
score: 1.79466395291
acc: 0.758865248227
Train on 6341 samples, validate on 705 samples
Epoch 1/100
6341/6341 [==============================] - 192s 30ms/step - loss: 0.8035 - acc: 0.5053 - val_loss: 0.7426 - val_acc: 0.5007
Epoch 2/100
6341/6341 [==============================] - 135s 21ms/step - loss: 0.7538 - acc: 0.5007 - val_loss: 0.7428 - val_acc: 0.5007
Epoch 3/100
6341/6341 [==============================] - 110s 17ms/step - loss: 0.7589 - acc: 0.5070 - val_loss: 0.7425 - val_acc: 0.5007
Epoch 4/100
6341/6341 [==============================] - 77s 12ms/step - loss: 0.7576 - acc: 0.4969 - val_loss: 0.7517 - val_acc: 0.4993
Epoch 5/100
6341/634

6341/6341 [==============================] - 147s 23ms/step - loss: 0.4981 - acc: 0.7732 - val_loss: 0.6976 - val_acc: 0.6950
Epoch 9/100
6341/6341 [==============================] - 146s 23ms/step - loss: 0.4899 - acc: 0.7821 - val_loss: 0.8589 - val_acc: 0.6355
Epoch 10/100
6341/6341 [==============================] - 153s 24ms/step - loss: 0.4920 - acc: 0.7825 - val_loss: 0.5759 - val_acc: 0.7262
Epoch 11/100
6341/6341 [==============================] - 152s 24ms/step - loss: 0.4978 - acc: 0.7773 - val_loss: 0.5928 - val_acc: 0.6993
Epoch 12/100
6341/6341 [==============================] - 153s 24ms/step - loss: 0.4837 - acc: 0.7805 - val_loss: 0.6125 - val_acc: 0.6525
Epoch 13/100
6341/6341 [==============================] - 153s 24ms/step - loss: 0.4762 - acc: 0.7903 - val_loss: 0.5802 - val_acc: 0.7163
Epoch 14/100
6341/6341 [==============================] - 156s 25ms/step - loss: 0.4722 - acc: 0.7846 - val_loss: 0.6470 - val_acc: 0.7333
Epoch 15/100
6341/6341 [=================

6341/6341 [==============================] - 36s 6ms/step - loss: 0.5895 - acc: 0.7349 - val_loss: 0.6034 - val_acc: 0.7248
Epoch 11/100
6341/6341 [==============================] - 38s 6ms/step - loss: 0.5846 - acc: 0.7448 - val_loss: 0.7403 - val_acc: 0.6879
Epoch 12/100
6341/6341 [==============================] - 36s 6ms/step - loss: 0.5979 - acc: 0.7294 - val_loss: 0.6612 - val_acc: 0.7121
Epoch 13/100
6341/6341 [==============================] - 36s 6ms/step - loss: 0.5914 - acc: 0.7431 - val_loss: 0.6025 - val_acc: 0.7291

Epoch 00013: ReduceLROnPlateau reducing learning rate to 0.0293366089463.
Epoch 14/100
6341/6341 [==============================] - 36s 6ms/step - loss: 0.5252 - acc: 0.7750 - val_loss: 0.6049 - val_acc: 0.7305
Epoch 15/100
6341/6341 [==============================] - 37s 6ms/step - loss: 0.5051 - acc: 0.7958 - val_loss: 0.6340 - val_acc: 0.7262
Epoch 16/100
6341/6341 [==============================] - 36s 6ms/step - loss: 0.4969 - acc: 0.8041 - val_loss: 0.60

In [5]:
x_train, y_train, x_valid, y_valid = data()
print("Evalutation of best performing model:")
print(best_model.evaluate(x_valid, y_valid))
print("Best performing model chosen hyper-parameters:")
print(best_run)

Evalutation of best performing model:
705/705 [==============================] - 7s 10ms/step
[0.511931981762924, 0.7943262411347518]
Best performing model chosen hyper-parameters:
{'dr_rate_0_1': 0.8010934391495235, 'dr_rate_0': 0.34691402700128554, 'batch_size': 1, 'l2': 0.0009996198898434667, 'lr': 0.001169490205654279, 'n_recurrent_2': 3, 'n_recurrent_1': 2, 'n_recurrent': 2, 'embedding_trainable': 1}


In [ ]:
best_model.summary()

In [6]:
def test_data():
    """Test set not returned on purpose."""
    
    labelsfile = '../../tmp/1_METONLY_vs_METX/matched_CEM_table.csv'
    datafile = '../../tmp/item_days_raw_data_.pkl'
    dataset = utils.load_data_labels(datafile, labelsfile)
    padded_mbs_seq, padded_timespan_seq, _ = utils.tokenize(dataset)
    maxlen = padded_mbs_seq.shape[1]

    # Split in training, validation, test sets
    _, _, ts_set = utils.train_validation_test_split(
                   [padded_mbs_seq, padded_timespan_seq], dataset['Class'],
                   test_size=0.4, validation_size=0.1,
                   verbose=False, random_state0=1, random_state1=0)
    
    return ts_set[0], ts_set[1]

In [7]:
print('Evaluate on the test set')
x_test, y_test = test_data()
y_pred = best_model.predict(x_test).ravel()

loss = metrics.log_loss(y_test, y_pred)
acc = metrics.accuracy_score(y_test, y_pred > 0.5)
prec = metrics.precision_score(y_test, y_pred > 0.5)
rcll = metrics.recall_score(y_test, y_pred > 0.5)
auc = metrics.roc_auc_score(y_test, y_pred)
print('Test scores:\n * Log-Loss\t{}\n * Accuracy:\t{}\n '
      '* Precision:\t{}\n * Recall:\t{}\n * AUC: \t{}'.format(loss, acc, prec, rcll, auc))

Evaluate on the test set
Test scores:
 * Log-Loss	nan
 * Accuracy:	0.837377607493
 * Precision:	0.861047835991
 * Recall:	0.804597701149
 * AUC: 	0.914321484229


/home/samu/miniconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1769: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/samu/miniconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1769: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
